In [ ]:

import json
import os
import openai
import jsonlines
from tqdm import tqdm
import time
import argparse

model_name = "gpt-4o-2024-08-06" # 
openai.api_key = ''


In [ ]:
path = "../../nlp_data/kisti/(분류) 국내 논문 전문 텍스트 데이터셋"
domains = os.listdir(path)
domains[:5], len(domains)

In [ ]:
domain2papers = {}
for domain in domains:
    years = os.listdir(path + "/" + domain)
    paper_paths = []
    for year in years:
        papers = os.listdir(path + "/" + domain + "/" + year)
        paper_paths += papers
        
    domain2papers[domain] = paper_paths

In [ ]:
domain2cnt = {domain:len(domain2papers[domain]) for domain in domains}
domain2cnt = dict(sorted(domain2cnt.items(), key=lambda x:-x[1]))
list(domain2cnt.items())[:100]

In [ ]:
sum(list(domain2cnt.values()))

In [ ]:
def count_korean_and_english(text):
    korean_count = 0
    english_count = 0
    
    for char in text:
        if '\uAC00' <= char <= '\uD7AF':
            korean_count += 1
        elif ('\u0041' <= char <= '\u005A') or ('\u0061' <= char <= '\u007A'):
            english_count += 1
    
    return korean_count, english_count


def count_korean_and_english_path(path):
    f = open(path)
    body = json.load(f,)['body_text']
    total_korean_count = 0
    total_english_count = 0
    for section in body:
        # if 'section' in section:
        #     korean_count += len(section['section'])
        if 'text' in section:
            for text in section['text']:
                korean_count, english_count = count_korean_and_english(text)
                total_korean_count += korean_count
                total_english_count += english_count
    return total_korean_count, total_english_count

                # korean_count += len(text)
total_korean_count, total_english_count = count_korean_and_english_path("../../nlp_data/kisti/(분류) 국내 논문 전문 텍스트 데이터셋/재무관리연구/2009/JAKO200933359739803.json")
total_korean_count, total_english_count

In [ ]:
paper2lang_cnt = {}
for domain in domains:
    years = os.listdir(path + "/" + domain)
    en_paper_paths = []
    for year in years:
        papers = os.listdir(path + "/" + domain + "/" + year)
        for p_path in papers:
            f_path = path + "/" + domain + "/" + year + "/" + p_path
            total_korean_count, total_english_count = count_korean_and_english_path(f_path)
            paper2lang_cnt[f_path] = (total_korean_count, total_english_count)
      

In [ ]:
en_papers = {}
for f_path, cnts in paper2lang_cnt.items():
    total_korean_count, total_english_count = cnts
    if total_korean_count+total_english_count>0 and total_english_count/(total_korean_count+total_english_count)> 0.99:
        en_papers[f_path] = cnts

In [ ]:
len(en_papers)

In [ ]:
list(paper2lang_cnt.keys())[:5]

In [ ]:
import torch
torch.save(en_papers, "../../nlp_data/kisti/db_files/en_paperpath2lang_cnt")
torch.save(paper2lang_cnt, "../../nlp_data/kisti/db_files/paperpath2lang_cnt")


In [ ]:
paper2lang_cnt['../../nlp_data/kisti/(분류) 국내 논문 전문 텍스트 데이터셋/천문학논총/2014/JAKO201428854482770.json']

In [ ]:
from collections import Counter
domain_en_papers = [p_path.split('/')[6] for p_path in list(en_papers.keys())]
domain_en_papers = Counter(domain_en_papers)
domain_en_papers.most_common(40)

In [ ]:
certain_domain_en_ps = [en_p_path for en_p_path in list(en_papers.keys()) if "아세아태평양축산학회지" in en_p_path] # 2576
# certain_domain_en_ps = [en_p_path for en_p_path in list(en_papers.keys()) if "한국발생생물학회지-발생과생식" in en_p_path] # 405
# certain_domain_en_ps = [en_p_path for en_p_path in list(en_papers.keys()) if "조선자연과학논문집" in en_p_path] # 347
# certain_domain_en_ps = [en_p_path for en_p_path in list(en_papers.keys()) if "천문학논총" in en_p_path] # 347
# certain_domain_en_ps = [en_p_path for en_p_path in list(en_papers.keys()) if "목재공학" in en_p_path] # 396
# certain_domain_en_ps = [en_p_path for en_p_path in list(en_papers.keys()) if "패션비즈니스" in en_p_path] # 
# certain_domain_en_ps = [en_p_path for en_p_path in list(en_papers.keys()) if "대한위암학회지" in en_p_path] # 409
# certain_domain_en_ps = [en_p_path for en_p_path in list(en_papers.keys()) if "한국전문물리치료학회지" in en_p_path] # 280

len(certain_domain_en_ps), certain_domain_en_ps[:5]

In [ ]:
def extract_content_path(path):
    f = open(path) # certain_domain_en_ps[5]
    # print(path)
    body = json.load(f,)['body_text']
    all_content = ""
    # print(body)
    prev_section_name = ""
    for section in body:

        if 'text' in section:
            section_name = section['section'] if 'section' in section else ""
            for text in section['text']:
                if prev_section_name:
                    # print(prev_section_name+": "+section_name+": "+ text)
                    all_content += prev_section_name+": "+section_name+": "+ text + "\n"
                else:
                    # print(section_name+": "+ text)
                    all_content += section_name+": "+ text + " "
            prev_section_name = ""
        else:
            prev_section_name = section['section']
    # print()
    # len(all_content)
    return all_content

long_enough_ps = [f_path for f_path in certain_domain_en_ps if len(extract_content_path(f_path))>15000]
len(long_enough_ps)

import random
random.seed(0)
sampled_papers = random.sample(long_enough_ps, 900)

In [ ]:
template = \
"""Create an information-seeking conversation between two annotators: a questioner and an answerer. We give you multiple sections of an academic paper as seed topics for the information-seeking questions. 
Assume that the questioner has access only to the main topic of the given content, while the answerer can access the full text. 
Allow topic switching by enabling the answerer to refer to sections from different sections of papers.

The annotators are provided with the following guidelines.
Guidelines for the questioner: 
    • The first question should be unambiguous and clear about the main topic of academic paper. The questioner, only knowing the main topic, cannot ask directly about the study's focus. Thus, do not ask like "What is the primary focus of the study?" as 1st question.
    • The follow-up questions are contextualized and always dependent on the conversation history (especially, last answerer's repond) so that question itself is hard to understand. 
    • Avoid using same words as in section titles of the document. E.g. if the section title is “Awards”, a plausible question can be “What accolades did she receive for her work?”. • The conversation should involve multiple documents (topics).
    
Guidelines for the answerer: 
    • Based on the question, identify the relevant document and section. 
    • The answer should be based on the contents of the identified document. 
    • The rationale should be a sub-string of content such that it justifies the answer and should be recorded below the answers.
    • The answer should be a sub-string in rationale whenever possible. However, answers should be edited to fit the conversational context (adding yes, no), perform reasoning (e.g. counting) etc. 
    • Personal opinions should never be included.

- Example of an information-seeking conversation given the content:
-- content: 
The Byzantine Empire, also referred to as the Eastern Roman Empire, was the continuation of the Roman Empire centered in Constantinople during Late Antiquity and the Middle Ages. The eastern half of the Empire survived the conditions that caused the fall of the West in the 5th century AD, and continued to exist until the fall of Constantinople to the Ottoman Empire in 1453. During most of its existence, the empire remained the most powerful economic, cultural, and military force in the Mediterranean world. The term "Byzantine Empire" was only coined following the empire's demise; its citizens referred to the polity as the "Roman Empire" and to themselves as "Romans".[a] Due to the imperial seat's move from Rome to Byzantium, the adoption of state Christianity, and the predominance of Greek instead of Latin, modern historians continue to make a distinction between the earlier Roman Empire and the later Byzantine Empire.
The empire was largely dismantled in 1204, following the Sack of Constantinople by Latin armies at the end of the Fourth Crusade; its former territories were then divided into competing Greek rump states and Latin realms. Despite the eventual recovery of Constantinople in 1261, the reconstituted empire would wield only regional power during its final two centuries of existence. Its remaining territories were progressively annexed by the Ottomans in perennial wars fought throughout the 14th and 15th centuries. The fall of Constantinople to the Ottomans in 1453 ultimately brought the empire to an end. Many refugees who had fled the city after its capture settled in Italy and throughout Europe, helping to ignite the Renaissance. The fall of Constantinople is sometimes used to mark the dividing line between the Middle Ages and the early modern period.
The situation became worse for Byzantium during the civil wars after Andronikos III died. A six-year-long civil war devastated the empire, allowing the Serbian ruler Stefan Dušan to overrun most of the empire's remaining territory and establish a Serbian Empire. In 1354, an earthquake at Gallipoli devastated the fort, allowing the Ottomans (who were hired as mercenaries during the civil war by John VI Kantakouzenos) to establish themselves in Europe.[115][116] By the time the Byzantine civil wars had ended, the Ottomans had defeated the Serbians and subjugated them as vassals. Following the Battle of Kosovo, much of the Balkans became dominated by the Ottomans.[117]
Constantinople by this stage was underpopulated and dilapidated. The population of the city had collapsed so severely that it was now little more than a cluster of villages separated by fields. On 2 April 1453, Sultan Mehmed's army of 80,000 men and large numbers of irregulars laid siege to the city.[118] Despite a desperate last-ditch defence of the city by the massively outnumbered Christian forces (c. 7,000 men, 2,000 of whom were foreign),[119] Constantinople finally fell to the Ottomans after a two-month siege on 29 May 1453. The final Byzantine emperor, Constantine XI Palaiologos, was last seen casting off his imperial regalia and throwing himself into hand-to-hand combat after the walls of the city were taken.[120]
Mehmed continued his conquests in Anatolia with its reunification and in Southeast Europe as far west as Bosnia. At home, he made many political and social reforms. He encouraged the arts and sciences, and by the end of his reign, his rebuilding program had changed Constantinople into a thriving imperial capital. He is considered a hero in modern-day Turkey and parts of the wider Muslim world. Among other things, Istanbul's Fatih district, Fatih Sultan Mehmet Bridge and Fatih Mosque are named after him.
Anatolia (Turkish: Anadolu), also known as Asia Minor,[a] is a large peninsula or a region in Turkey, constituting most of its contemporary territory. Geographically, the Anatolian region is bounded by the Mediterranean Sea to the south, the Aegean Sea to the west, the Turkish Straits to the north-west, and the Black Sea to the north. The eastern and southeastern boundary is either the southeastern and eastern borders of Turkey,[1] or an imprecise line from the Black Sea to Gulf of Iskenderun.[2] Topographically, the Sea of Marmara connects the Black Sea with the Aegean Sea through the Bosporus strait and the Dardanelles strait, and separates Anatolia from Thrace in the Balkan peninsula of Southeastern Europe.
The Akkadian Empire (/əˈkeɪdiən/)[2] was the first known ancient empire of Mesopotamia, succeeding the long-lived civilization of Sumer. Centered on the city of Akkad (/ˈækæd/)[3] and its surrounding region, the empire united Akkadian and Sumerian speakers under one rule and exercised significant influence across Mesopotamia, the Levant, and Anatolia, sending military expeditions as far south as Dilmun and Magan (modern United Arab Emirates, Saudi Arabia, Bahrain, Qatar and Oman) in the Arabian Peninsula.[4][page needed]

-- Information-seeking conversation
Q1: when was the byzantine empire born what was it originally called?
A1: 5th century AD and was called Eastern Roman Empire, or Byzantium
rationale: The Byzantine Empire, also referred to as the Eastern Roman Empire, was the continuation of the Roman Empire centered in Constantinople during Late Antiquity and the Middle Ages. The eastern half of the Empire survived the conditions that caused the fall of the West in the 5th century AD, and continued to exist until the fall of Constantinople to the Ottoman Empire in 1453.
                                                                                                   
Q2: and when did it fall?
A2: 1453
rationale: The fall of Constantinople to the Ottomans in 1453 ultimately brought the empire to an end. 
                                                                                                   
Q3: which battle or event marked the fall of this empire?
A3: A six-year-long civil war followed by attack from Sultan Mehmed’s army
rationale: A six-year-long civil war devastated the empire;  On 2 April 1453, Sultan Mehmed's army of 80,000 men and large numbers of irregulars laid siege to the city.
                                                                                                   
Q4: did he conquer other territories as well?
A4: Yes. Anatolia and in Southeast Europe as far west as Bosnia
rationale: Mehmed continued his conquests in Anatolia with its reunification and in Southeast Europe as far west as Bosnia
                                                                                                   
Q5: where is the first area located in present day terms?
A5: Turkey
rationale: Anatolia (Turkish: Anadolu), also known as Asia Minor,[a] is a large peninsula or a region in Turkey
                                                                                                   
Q6: who were the oldest known inhabitants of this region?
A6: Mesopotamian-based Akkadian Empire
rationale: The Akkadian Empire (/əˈkeɪdiən/)[2] was the first known ancient empire of Mesopotamia

- sections of academic papers:
content: """
                
last_inst = """Write an information-seeking conversation with roughly ten to fifteen QA turns based on the given sections of academic paper above, following the format of the above example provided. 
Except for the first question, remember that questions should always be contextualized and based on the conversation history. For example, to ensure the question is challenging to understand without the conversation history, it should use coreferences rather than "proper nouns," such as their specific identifiers such as names or titles or country names.
Do not use content from an "example of an information-seeking conversation" when making questions.
"""
                                                                                                   

In [ ]:
def generate_conv_turns(path):
    # sampled_papers = random.sample(long_enough_ps, 100)
    all_content = extract_content_path(path) # sampled_papers[1]

    prompt = f"{template}\n{all_content[:10000]}\n\n{last_inst}"

    # print(prompt)
    # prompt = set_prompt(line, args)

    messages = [
        {"role": "user", "content": prompt}
    ]

    retries = 5
    delay = 1
    while retries > 0:
        try:
            response = openai.ChatCompletion.create(
                model=model_name,
                messages=messages,
                # temperature=0.8, # deterministic decoding: 0.
                # max_tokens=2560,
                # top_p=0.8, # deterministic decoding: 1.
                n=1, # number of output
            )
            break
            # return response
        except:
            pass
        retries -= 1
        time.sleep(delay)
        delay *= 2

    # print(response["choices"][0]["message"]["content"])
    return response


In [ ]:
paperpath2conv = {}
for sampled_path in sampled_papers:# [:1]: # [:2]:
    paperpath2conv[sampled_path] = generate_conv_turns(sampled_path)
    
# paperpath2conv
key = list(paperpath2conv.keys())[0]
print(paperpath2conv[key]["choices"][0]["message"]["content"])

In [ ]:

torch.save(paperpath2conv, "../../nlp_data/kisti/db_files/paperpath2gen_conv")


### post-processing conv: paraphrasing responses

In [ ]:
gen_paperpath2As = torch.load("../../nlp_data/kisti/db_files/gen_paperpath2As")
gen_paperpath2As[list(gen_paperpath2As.keys())[0]]

In [ ]:
template = "The following text is a response from an AI chatbot. Please retain the exact meaning of the subsequent sentence but rephrase it as much as possible, ensuring that the key terms remain unchanged."
def paraphrase_response(response):
    # sampled_papers = random.sample(long_enough_ps, 100)
    # all_content = extract_content_path(path) # sampled_papers[1]

    prompt = f"{template}\n{response}"

    # print(prompt)
    # prompt = set_prompt(line, args)

    messages = [
        {"role": "user", "content": prompt}
    ]

    retries = 5
    delay = 1
    while retries > 0:
        try:
            response = openai.ChatCompletion.create(
                model=model_name,
                messages=messages,
                # temperature=0.8, # deterministic decoding: 0.
                # max_tokens=2560,
                # top_p=0.8, # deterministic decoding: 1.
                n=1, # number of output
            )
            break
            # return response
        except:
            pass
        retries -= 1
        time.sleep(delay)
        delay *= 2

    # print(response["choices"][0]["message"]["content"])
    return response

# response = 'Yes, the infrared thermographic (IRT) technique can be used effectively for non-invasive monitoring of body surface temperature in the field.'
# paraphrase_response(response)

In [ ]:
gen_paperpath2newAs = {}
   
paths = list(gen_paperpath2As.keys())
for path in paths:
    responses = gen_paperpath2As[path]
    new_responses = []
    for resp in responses:
        new_resp = paraphrase_response(resp)
        new_resp = new_resp["choices"][0]["message"]["content"]
        new_responses += [new_resp]
    gen_paperpath2newAs[path] = new_responses

In [ ]:
torch.save(gen_paperpath2newAs, "../../nlp_data/kisti/db_files/gen_paperpath2newAs")
